In [11]:
pip install numpy pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib
from scipy.stats import fisher_exact
# control the working directory
print(os.getcwd())

# data import and pandas manipulation
fp_savi = "../input.savi.txt"
cutoff_refdepth_B = 20
cutoff_altdepth_B = 1
cutoff_freq = 5

# function 1 
raw_data = pd.read_csv(fp_savi, delimiter="\t", index_col=False)
#print(raw_data.describe())

df_savi = raw_data.loc[lambda df:(df["refdepth_Blood"] >= cutoff_refdepth_B) & 
                                 (df["altdepth_Blood"] <= cutoff_altdepth_B) & 
                                 (df["Sgt1_max_frequency"] >= cutoff_freq), :]
df_savi = df_savi.fillna(0)


/Users/jihongtang/Documents/GitHub/CELLO/CELLOP


In [13]:
# Cell for function mutStats preparation 
# function 2 
df_savi = df_savi
genelist_selected = ['LTBP4', 'PTPN11', 'NF1', 'RB1', 'PDGFRA','PIK3CG', 'PIK3R1', 'PIK3CA', 'PTEN', 'EGFR', 'IDH1', 'ATRX', 'TP53']
cutoff_freq = 5
remove_LOW = True

# remove the synonymous variant labelled by LOW in feature Effect_Impact
if(remove_LOW):
    df_savi = df_savi.loc[lambda df: df["Effect_Impact"] != 'LOW', :]

# pandas Series object ==> list data structure
case = df_savi["CaseID"].drop_duplicates().tolist()

# counting different mutations
mut_P = [0] * len(case)
mut_R = [0] * len(case)
mut_C = [0] * len(case)

# use lambda function to count mutations in different type
for i in range(len(case)):
    mut_C[i] = (lambda df: (df["CaseID"] == case[i]) 
                            & (df["Primary_freq"] >= cutoff_freq) 
                            & (df["Recurrent_freq"] >= cutoff_freq)
                )(df_savi).sum()
    mut_P[i] = (lambda df: (df["CaseID"] == case[i]) 
                            & (df["Primary_freq"] >= cutoff_freq) 
                            & (df["Recurrent_freq"] < cutoff_freq)
                )(df_savi).sum()
    mut_R[i] = (lambda df: (df["CaseID"] == case[i]) 
                            & (df["Primary_freq"] < cutoff_freq) 
                            & (df["Recurrent_freq"] >= cutoff_freq)
                )(df_savi).sum()

df_mut_num = pd.DataFrame({'Patient': case,
                            'Primary': mut_P,
                            'Common': mut_C,
                            'Recurrent': mut_R})


In [14]:
# create 2D list in python using following format to make sure to create several 
# separate lists in the second dimension
list_mut_gene = [['N'] * len(genelist_selected) for _ in range(len(case))]
df_savi_gene = df_savi[df_savi.Gene_Name.isin(genelist_selected)]

geneSel = genelist_selected
for i in range(len(case)):
    for j in range(len(geneSel)):
        temp_P = df_savi_gene.Primary_freq[(df_savi_gene.CaseID == case[i]) & (df_savi_gene.Gene_Name == geneSel[j])]
        temp_R = df_savi_gene.Recurrent_freq[(df_savi_gene.CaseID == case[i]) & (df_savi_gene.Gene_Name == geneSel[j])]
        if any(temp_P >= cutoff_freq) & any(temp_R >= cutoff_freq): 
            list_mut_gene[i][j] = 'C'
        elif any(temp_P >= cutoff_freq):
            list_mut_gene[i][j] = 'P'
        elif any(temp_R >= cutoff_freq):
            list_mut_gene[i][j] = 'R'
        else:
            list_mut_gene[i][j] = 'N'

df_mut_gene = pd.DataFrame(np.array(list_mut_gene), columns=geneSel)
df_mut_gene.index = case


In [15]:
# funciton 3 


In [16]:
# function 4
# co-mutation analysis 
df_mut_gene = df_mut_gene
cutoff_pValue = 0.1
# use replace function and regular expression to generate Primary and Recurrent pattern matrix
mt_mut_P = df_mut_gene.replace(
    r'[C, P]', 1, regex=True).replace(r'[R, N]', 0, regex=True)
mt_mut_R = df_mut_gene.replace(
    r'[C, R]', 1, regex=True).replace(r'[P, N]', 0, regex=True)

# prepare the data of co-mutation analysis
list_columns = list(df_mut_gene.columns)
plot_mutCor = [[0] * 4 for _ in range(len(list_columns)**2)]
idx = 0
for i in range(len(df_mut_gene.columns)):
    for j in range(len(df_mut_gene.columns)):
        plot_mutCor[idx][0] = list_columns[i]
        plot_mutCor[idx][1] = list_columns[j]

        mt_FEtest = np.array([[0,0], [0,0]]) # initial the matrix for fisher exact test
        if i > j: # primary tumor section
            mt_FEtest[0][0] = sum((mt_mut_P.iloc[:,i] == 1)
                                  & (mt_mut_P.iloc[:,j] == 1))
            mt_FEtest[0][1] = sum((mt_mut_P.iloc[:,i] == 1)
                                  & (mt_mut_P.iloc[:,j] == 0))
            mt_FEtest[1][0] = sum((mt_mut_P.iloc[:,i] == 0)
                                  & (mt_mut_P.iloc[:,j] == 1))
            mt_FEtest[1][1] = sum((mt_mut_P.iloc[:,i] == 0)
                                  & (mt_mut_P.iloc[:,j] == 0))
            idx_coMutation = ((mt_FEtest[0][0]+1) * (mt_FEtest[1][1]+1)) / \
                ((mt_FEtest[0][1]+1) * (mt_FEtest[1][0]+1))
            
            # fisher exact test
            odds, pValue = fisher_exact(mt_FEtest, alternative='two-sided')

            if pValue < cutoff_pValue:
                plot_mutCor[idx][2] = -np.log10(pValue) + 1 # for dot size, if the pvalue is lager than cutoff value, fill in instinct value
                if idx_coMutation > 1:
                    plot_mutCor[idx][3] = 'D_red'
                else:
                    plot_mutCor[idx][3] = 'A_blue'
            else:
                plot_mutCor[idx][2] = 1
                plot_mutCor[idx][3] = 'C_grey'

        elif i < j: # recurrent tumor section
            mt_FEtest[0][0] = sum((mt_mut_R.iloc[:, i] == 1)
                                  & (mt_mut_R.iloc[:, j] == 1))
            mt_FEtest[0][1] = sum((mt_mut_R.iloc[:,i] == 1)
                                  & (mt_mut_R.iloc[:,j] == 0))
            mt_FEtest[1][0] = sum((mt_mut_R.iloc[:,i] == 0)
                                  & (mt_mut_R.iloc[:,j] == 1))
            mt_FEtest[1][1] = sum((mt_mut_R.iloc[:,i] == 0)
                                  & (mt_mut_R.iloc[:,j] == 0))
            idx_coMutation = ((mt_FEtest[0][0]+1) * (mt_FEtest[1][1]+1)) / \
                ((mt_FEtest[0][1]+1) * (mt_FEtest[1][0]+1))
            
            # fisher exact test
            odds, pValue = fisher_exact(mt_FEtest, alternative='two-sided')

            if pValue < cutoff_pValue:
                plot_mutCor[idx][2] = -np.log10(pValue) + 1 # for dot size, if the pvalue is lager than cutoff value, fill in instinct value
                if idx_coMutation > 1:
                    plot_mutCor[idx][3] = 'E_black'
                else:
                    plot_mutCor[idx][3] = 'B_green'
            else:
                plot_mutCor[idx][2] = 1
                plot_mutCor[idx][3] = 'C_grey'
        
        else: # i==j, fill all with 'NA'
            plot_mutCor[idx][2] = np.NaN
            plot_mutCor[idx][3] = np.NaN
        
        idx += 1 # add the index for plot table 

df_mutCorMatrix = pd.DataFrame(np.array(plot_mutCor), columns=['listA', 'listB', 'dotSize', 'dotColor'])


In [17]:
# function 4 visulization


In [18]:
# function 5 3-D bubble plot of mutation frequency
# data matrix generation
df_savi = df_savi
genelist_selected = genelist_selected
df_mut_gene = df_mut_gene
cutoff_freq = 5

# remove the synonymous variant labelled by LOW in feature Effect_Impact
df_savi = df_savi.loc[lambda df: df["Effect_Impact"] != 'LOW', :]
# pandas Series object ==> list data structure
case = df_savi["CaseID"].drop_duplicates().tolist()

# 13 * 3 matrix initial
mt_mutFreq = [[0] * 3 for _ in range(len(genelist_selected))]
for i in range(len(genelist_selected)):
    df_savi_gene = df_savi[df_savi.Gene_Name == genelist_selected[i]]
    mt_mutFreq[i][0] = len(df_savi_gene.CaseID[(df_savi_gene.Primary_freq >= cutoff_freq) & (
        df_savi_gene.Recurrent_freq < cutoff_freq)].drop_duplicates())
    mt_mutFreq[i][1] = len(df_savi_gene.CaseID[(df_savi_gene.Primary_freq < cutoff_freq) & (
        df_savi_gene.Recurrent_freq >= cutoff_freq)].drop_duplicates())
    mt_mutFreq[i][2] = sum(df_mut_gene.iloc[:, i] == 'C')

df_mutFreq = pd.DataFrame(np.array(mt_mutFreq), columns=[
                          'Primary', 'Recurrent ', 'Common'])
df_mutFreq.index = genelist_selected

print(df_mutFreq)

        Primary  Recurrent   Common
LTBP4         0           9       0
PTPN11        1           2       2
NF1           4          12      10
RB1           1           2       2
PDGFRA        2           3       1
PIK3CG        2           1       5
PIK3R1        1           3       3
PIK3CA        1           4       6
PTEN          5          10      15
EGFR          9           5       9
IDH1          0           1      18
ATRX          5           4      16
TP53          8          13      31


In [19]:
# function 5 - 3D bubble plot visulization


In [53]:
# function 6 - HM analysis
# 4a data
cutoff_mut_freq = 15
cutoff_mut_num = 300
cutoff_HM_score = 1.3

# initilize the parameters 
cutoff_varPresent = cutoff_mut_freq
cutoff_mutLoad = cutoff_mut_num
cutoff_HMscore = cutoff_HM_score

case = df_savi.CaseID.drop_duplicates().tolist()

# initilize all the useful list objects
mutLoad_P = [0] * len(case)
mutLoad_R = [0] * len(case)
HMscore_P = [0] * len(case)
HMscore_R = [0] * len(case)
savi_P = np.array([0] * len(df_savi)) # use numpy arry for futher multiple change 
savi_R = np.array([0] * len(df_savi)) # use numpy arry for futher multiple change 
savi_isHM = np.array([0] * len(df_savi))

# assign three features to gain the new HM df for further calculation
df_savi_HM = df_savi.assign(isC2T=lambda df: 
                            ((df.ref == 'C') & (df.alt == 'T')) | 
                            ((df.ref == 'G') & (df.alt == 'A')))
df_savi_HM = df_savi_HM.assign(isCC2TC=lambda df:
                            ((df.ref == 'C') & (df.alt == 'T') & (df.varSurffix == 'C')) |
                            ((df.ref == 'G') & (df.alt == 'A') & (df.varSurffix == 'G')))                            
df_savi_HM = df_savi_HM.assign(isCT2TT=lambda df:
                               ((df.ref == 'C') & (df.alt == 'T') & (df.varSurffix == 'T')) |
                               ((df.ref == 'G') & (df.alt == 'A') & (df.varSurffix == 'A')))
df_savi_HM.index = range(len(df_savi_HM))
psedcount = 1 # parameter for HM score calculation

for i in range(len(case)): # for each patient 
    df_patient = df_savi_HM[df_savi_HM.CaseID == case[i]]
    
    # primary 
    df_varP = df_patient[(df_patient.Blood_freq == 0) & 
                         (df_patient.Primary_freq > cutoff_varPresent)]
    num_varP = len(df_varP)
    savi_P[df_varP.index.values] = [1] * len(df_varP) # numpy array benefits here
    mutLoad_P[i] = num_varP

    # HM score calculation part IMPORTANT
    pC2T = sum(df_varP.isC2T) / (mutLoad_P[i] + psedcount)
    pCC2TC = sum(df_varP.isCC2TC) / (sum(df_varP.isC2T) + psedcount)
    pCT2TT = sum(df_varP.isCT2TT) / (sum(df_varP.isC2T) + psedcount)
    # IMPORTANT
    HMscore_P[i] = pC2T + pCC2TC + np.sign(pCC2TC - pCT2TT) * pCT2TT

    # recurrent
    df_varR = df_patient[(df_patient.Blood_freq == 0) &
                         (df_patient.Recurrent_freq > cutoff_varPresent)]
    num_varR = len(df_varR)
    savi_R[df_varR.index.values] = [1] * len(df_varR)  # numpy array benefits here
    mutLoad_R[i] = num_varR

    # HM score calculation part IMPORTANT
    rC2T = sum(df_varR.isC2T) / (mutLoad_R[i] + psedcount)
    rCC2TC = sum(df_varR.isCC2TC) / (sum(df_varR.isC2T) + psedcount)
    rCT2TT = sum(df_varR.isCT2TT) / (sum(df_varR.isC2T) + psedcount)
    # IMPORTANT
    HMscore_R[i] = rC2T + rCC2TC + np.sign(rCC2TC - rCT2TT) * rCT2TT

HMmark_P = [0] * len(case)
HMmark_R = [0] * len(case)


for i in range(len(case)):
    if (mutLoad_P[i] > cutoff_mutLoad) & (HMscore_P[i] > cutoff_HMscore):
        savi_isHM[df_savi_HM.CaseID == case[i]] = 1
        HMmark_P[i] = 1
    
    if (mutLoad_R[i] > cutoff_mutLoad) & (HMscore_R[i] > cutoff_HMscore):
        savi_isHM[df_savi_HM.CaseID == case[i]] = 1
        HMmark_R[i] = 1

# temporary list to gain the result DataFrame
tmp_list = []
tmp_list.append(case * 2) # CaseID
tmp_arr1 = np.array(mutLoad_P) + 1
tmp_arr2 = np.array(mutLoad_R) + 1
tmp_list.append(np.concatenate((tmp_arr1, tmp_arr2))) # mutation number
tmp_list.append(HMscore_P + HMscore_R) # HM score
tmp_list.append(HMmark_P + HMmark_R) # HM mark 
tmp_list.append(['P'] * len(case) + ['R'] * len(case)) # PR mark

df_HM_Detection = pd.DataFrame(np.array(tmp_list).T, columns=
                               ['caseID', 'mutNumber', 'HMscore', 'HMmark', 'PRmark'])


In [ ]:
# function 6 HM analysis
# 4a plot

In [77]:
# function 6 HM analysis
# 4b data 
subTypes_P = df_savi_HM[savi_P == 1]
subTypes_R = df_savi_HM[(savi_isHM == 0) & (savi_R == 1)]
subTypes_HR = df_savi_HM[(savi_isHM == 1) & (savi_R == 1)]

# for primary
mut_CTGA_P = sum(((subTypes_P.ref == 'C') & (subTypes_P.alt == 'T')) |
                 ((subTypes_P.ref == 'G') & (subTypes_P.alt == 'A')))
mut_CGGC_P = sum(((subTypes_P.ref == 'C') & (subTypes_P.alt == 'G')) |
                 ((subTypes_P.ref == 'G') & (subTypes_P.alt == 'C')))
mut_CAGT_P = sum(((subTypes_P.ref == 'C') & (subTypes_P.alt == 'A')) |
                 ((subTypes_P.ref == 'G') & (subTypes_P.alt == 'T')))
mut_ATTA_P = sum(((subTypes_P.ref == 'A') & (subTypes_P.alt == 'T')) |
                 ((subTypes_P.ref == 'T') & (subTypes_P.alt == 'A')))
mut_AGTC_P = sum(((subTypes_P.ref == 'A') & (subTypes_P.alt == 'G')) |
                 ((subTypes_P.ref == 'T') & (subTypes_P.alt == 'C')))
mut_ACTG_P = sum(((subTypes_P.ref == 'A') & (subTypes_P.alt == 'C')) |
                 ((subTypes_P.ref == 'T') & (subTypes_P.alt == 'G')))
sumMut_P = sum([mut_CTGA_P, mut_CGGC_P, mut_CAGT_P, 
               mut_ATTA_P, mut_AGTC_P, mut_ACTG_P])


# for recurrent without hypermutation
mut_CTGA_R = sum(((subTypes_R.ref == 'C') & (subTypes_R.alt == 'T')) |
                 ((subTypes_R.ref == 'G') & (subTypes_R.alt == 'A')))
mut_CGGC_R = sum(((subTypes_R.ref == 'C') & (subTypes_R.alt == 'G')) |
                 ((subTypes_R.ref == 'G') & (subTypes_R.alt == 'C')))
mut_CAGT_R = sum(((subTypes_R.ref == 'C') & (subTypes_R.alt == 'A')) |
                 ((subTypes_R.ref == 'G') & (subTypes_R.alt == 'T')))
mut_ATTA_R = sum(((subTypes_R.ref == 'A') & (subTypes_R.alt == 'T')) |
                 ((subTypes_R.ref == 'T') & (subTypes_R.alt == 'A')))
mut_AGTC_R = sum(((subTypes_R.ref == 'A') & (subTypes_R.alt == 'G')) |
                 ((subTypes_R.ref == 'T') & (subTypes_R.alt == 'C')))
mut_ACTG_R = sum(((subTypes_R.ref == 'A') & (subTypes_R.alt == 'C')) |
                 ((subTypes_R.ref == 'T') & (subTypes_R.alt == 'G')))
sumMut_R = sum([mut_CTGA_R, mut_CGGC_R, mut_CAGT_R,
               mut_ATTA_R, mut_AGTC_R, mut_ACTG_R])

# for recurrent with hypermutation
mut_CTGA_HR = sum(((subTypes_HR.ref == 'C') & (subTypes_HR.alt == 'T')) |
                 ((subTypes_HR.ref == 'G') & (subTypes_HR.alt == 'A')))
mut_CGGC_HR = sum(((subTypes_HR.ref == 'C') & (subTypes_HR.alt == 'G')) |
                 ((subTypes_HR.ref == 'G') & (subTypes_HR.alt == 'C')))
mut_CAGT_HR = sum(((subTypes_HR.ref == 'C') & (subTypes_HR.alt == 'A')) |
                 ((subTypes_HR.ref == 'G') & (subTypes_HR.alt == 'T')))
mut_ATTA_HR = sum(((subTypes_HR.ref == 'A') & (subTypes_HR.alt == 'T')) |
                 ((subTypes_HR.ref == 'T') & (subTypes_HR.alt == 'A')))
mut_AGTC_HR = sum(((subTypes_HR.ref == 'A') & (subTypes_HR.alt == 'G')) |
                 ((subTypes_HR.ref == 'T') & (subTypes_HR.alt == 'C')))
mut_ACTG_HR = sum(((subTypes_HR.ref == 'A') & (subTypes_HR.alt == 'C')) |
                 ((subTypes_HR.ref == 'T') & (subTypes_HR.alt == 'G')))
sumMut_HR = sum([mut_CTGA_HR, mut_CGGC_HR, mut_CAGT_HR,
               mut_ATTA_HR, mut_AGTC_HR, mut_ACTG_HR])

tmp_list = []
tmp_list.append(['Primary'] * 6 + ['Rec_noHM'] * 6 + ['Rec_HM'] * 6)
tmp_frac = np.array(
           [mut_CTGA_P, mut_CGGC_P, mut_CAGT_P, 
            mut_ATTA_P, mut_AGTC_P, mut_ACTG_P] + \
           [mut_CTGA_R, mut_CGGC_R, mut_CAGT_R,
            mut_ATTA_R, mut_AGTC_R, mut_ACTG_R] + \
           [mut_CTGA_HR, mut_CGGC_HR, mut_CAGT_HR, 
            mut_ATTA_HR, mut_AGTC_HR, mut_ACTG_HR]
            )
tmp_sum = np.array([sumMut_P] * 6 + [sumMut_R] * 6 + [sumMut_HR] * 6)
tmp_list.append(tmp_frac / tmp_sum)
tmp_list.append(['CTGA', 'CGGC', 'CAGT', 'ATTA', 'AGTC', 'ACTG'] * 3)
df_HM_mutType = pd.DataFrame(np.array(tmp_list).T, columns=[
                             'sampleType', 'Fraction', 'MutType'])


In [ ]:
# function 6 HM analysis 
# 4b plot

In [ ]:
# function 6 HM analysis
# 4c data
